In [1]:
!pip3 install -i https://pypi.tuna.tsinghua.edu.cn/simple -r requirements.txt
!apt updateS
!apt install -y libsm6 libfontconfig1 libxrender1 libxext6  libgl1-mesa-glx

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the 'pip install --upgrade pip' command.
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease                  
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:5 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1392 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [50.5 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2163 kB]mm
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2163 kB]3m
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2163 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1392 kB]3m
Get:8 http://s

Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1963 kB]33m
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1963 kB]33m
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1963 kB]33m
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1963 kB]33m
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [304 kB]3m
Ign:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packagesm
Get:8 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [43.5 kB]33m
Get:8 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [43.5 kB]3m
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [304 kB]3m
Ign:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packagesm
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [234 kB]
Fetched

In [5]:
import numpy as np
np.random.seed(111)
import os
import json
from yolo.frontend import create_yolo, get_object_labels

In [6]:
def setup_training(config_file):
    """make directory to save weights & its configuration """
    import shutil
    
    with open(config_file) as config_buffer:
        config = json.loads(config_buffer.read())
    dirname = config['train']['saved_folder']
    
    if os.path.isdir(dirname):
        print("{} is already exists. Weight file in directory will be overwritten".format(dirname))
    else:
        print("{} is created.".format(dirname, dirname))
        os.makedirs(dirname)
    
    print("Weight file and Config file will be saved in \"{}\"".format(dirname))
    shutil.copyfile(config_file, os.path.join(dirname, "fire.json"))
    """权重文件"""
    return config, os.path.join(dirname, "weights.h5")

In [7]:
import warnings
"""配置文件"""
config_file = "configs/fire.json"
config, weight_file = setup_training(config_file)

if config['train']['is_only_detect']:
    labels = ["object"]
else:
    if config['model']['labels']:
        labels = config['model']['labels']
    else:
        labels = get_object_labelsget_object_labels(config['train']['train_annot_folder'])
        
print(labels)

# 1. Construct the model 
yolo = create_yolo(config['model']['architecture'],
                   labels,
                   config['model']['input_size'],
                   config['model']['anchors'],
                   config['model']['coord_scale'],
                   config['model']['class_scale'],
                   config['model']['object_scale'],
                   config['model']['no_object_scale'])

# 2. Load the pretrained weights (if any) 
yolo.load_weights(config['pretrained']['full'], by_name=True)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # 3. actual training 
    yolo.train(config['train']['train_image_folder'],
               config['train']['train_annot_folder'],
               config['train']['actual_epoch'],
               weight_file,
               config["train"]["batch_size"],
               config["train"]["jitter"],
               config['train']['learning_rate'], 
               config['train']['train_times'],
               config['train']['valid_times'],
               config['train']['train_image_folder'],
               config['train']['train_annot_folder'],
               config['train']['first_trainable_layer'],
               config['train']['is_only_detect'])


fire is already exists. Weight file in directory will be overwritten
Weight file and Config file will be saved in "fire"
['fire']
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenet_0.75_224 (Model)   (None, 7, 7, 768)         1832976   
_________________________________________________________________
detection_layer_30 (Conv2D)  (None, 7, 7, 30)          23070     
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 5, 6)        0         
Total params: 1,856,046
Trainable params: 1,839,630
Non-trainable params: 16,416
_________________________________________________________________
Loading pre-trained weights in fire_3.h5
Epoch 1/30
393/393 [==============================] - 81s 206ms/step - 